In [1]:
####################################################################
# Scenario0: static하게 앙상블
# 모든 컴비네이션으로 경우의 수를 완탐해보고 마지막 출구보다 성능 좋은것 유무
####################################################################
# Scenario1: Entropy vs Temperature Scaling + Entropy 
# make dynamic ensemble of models which entropy is less than threshold
####################################################################
# Scenario2: MC Dropout -> find confident EE -> static ensemble 
#step1: make MC Dropout model
#step2: find confident EE from experiment
#step3: sum of softmax vector of each good model(under threshold) -> final inference from softmax vector sum
####################################################################
# Scenario3: train new block to choose which exit to inference 
# (JUST SCENARIO, NOT VERIFIED)
####################################################################

##### 모델 로드 및 캐시 저장

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import models,datasets, transforms
from mevit_model import MultiExitViT
from tqdm import tqdm
from itertools import combinations
import numpy as np
from scipy.stats import entropy
from collections import defaultdict
from temperature_scaling import TemperatureScaling
from Dloaders import Dloaders

IMG_SIZE = 224
device = 'cuda' if torch.cuda.is_available() else 'cpu'
dataset_name = {'cifar10':datasets.CIFAR10, 'cifar100':datasets.CIFAR100,'imagenet':None}
dataset_outdim = {'cifar10':10, 'cifar100':100,'imagenet':1000}
################ 0. Hyperparameters ##########################
batch_size = 1024
data_choice='cifar100'
exit_num=11
cache_file_path = f'cache_result_mevit_{data_choice}.pt'

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
dloaders=Dloaders(data_choice=data_choice,batch_size=batch_size,IMG_SIZE=IMG_SIZE)
train_loader,test_loader = dloaders.get_loaders()

# 저장한 파일을 다시 불러오기
output_tensor = torch.load(cache_file_path).to(device)
_, test_dataset = dloaders.get_datasets()
labels_list = test_dataset.targets
labels=torch.tensor(labels_list).to(device)
# 데이터 확인
print(output_tensor.shape)  # <class 'list'>
print(output_tensor[0].shape)  # torch.Size([10000, 100])

torch.Size([11, 10000, 100])
torch.Size([10000, 100])


##### 모든 경우의 수에 대해서 소프트맥스 전에 정적 앙상블 하고 소프트맥스 진행

In [4]:
combinations_list = []
for i in range(2, exit_num+1):combinations_list.extend(combinations(range(exit_num), i))

In [5]:
# sum before softmax (best acc: 87.75)
bef_softmax = dict()
for choosed_exits in combinations_list:
    new_output_list = output_tensor[choosed_exits,:,:]
    ensemble_logits = new_output_list.sum(axis=0)
    ensemble_probabilities = F.softmax(ensemble_logits, dim=1)
    _, predicted_labels = torch.max(ensemble_probabilities, dim=1)
    correct_predictions = (predicted_labels == labels).sum().item()
    total_predictions = labels.size(0)
    accuracy = correct_predictions / total_predictions * 100
    bef_softmax[choosed_exits]=accuracy

sorted(list(bef_softmax.items()), key=lambda x: x[1], reverse=True)

[((3, 7, 10), 88.57000000000001),
 ((3, 7, 9, 10), 88.5),
 ((2, 6, 7, 8, 9, 10), 88.47),
 ((2, 6, 7, 8, 10), 88.46000000000001),
 ((2, 7, 8, 9, 10), 88.46000000000001),
 ((3, 6, 7, 8, 9, 10), 88.46000000000001),
 ((5, 7, 9, 10), 88.44999999999999),
 ((4, 6, 7, 8, 9, 10), 88.44999999999999),
 ((4, 6, 10), 88.42999999999999),
 ((3, 7, 8, 10), 88.42999999999999),
 ((4, 6, 9, 10), 88.42999999999999),
 ((5, 6, 9, 10), 88.42999999999999),
 ((2, 7, 9, 10), 88.41),
 ((2, 6, 7, 9, 10), 88.41),
 ((0, 6, 7, 9, 10), 88.4),
 ((2, 4, 6, 9, 10), 88.38000000000001),
 ((3, 5, 7, 8, 9, 10), 88.38000000000001),
 ((5, 7, 10), 88.37),
 ((1, 6, 9, 10), 88.37),
 ((3, 4, 7, 8, 9, 10), 88.37),
 ((1, 6, 7, 10), 88.36),
 ((4, 6, 8, 10), 88.36),
 ((2, 3, 6, 7, 9, 10), 88.36),
 ((2, 5, 6, 7, 9, 10), 88.36),
 ((4, 5, 7, 8, 9, 10), 88.36),
 ((2, 7, 8, 10), 88.35),
 ((5, 7, 8, 10), 88.35),
 ((6, 7, 9, 10), 88.35),
 ((3, 7, 8, 9, 10), 88.35),
 ((2, 3, 7, 8, 9, 10), 88.35),
 ((2, 4, 7, 8, 9, 10), 88.35),
 ((2, 3, 6, 7,

##### 모든 경우의 수에 대해서 소프트맥스 하고 정적 앙상블 하고 맥스값으로 추론 진행

In [6]:
# sum after softmax (best acc: 87.75)
aft_softmax = dict()
for choosed_exits in combinations_list:
    new_output_list = output_tensor[choosed_exits,:,:]
    softmax_vector_list = F.softmax(new_output_list, dim=2)
    ensemble_probabilities = softmax_vector_list.sum(axis=0)
    _, predicted_labels = torch.max(ensemble_probabilities, dim=1)
    correct_predictions = (predicted_labels == labels).sum().item()
    total_predictions = labels.size(0)
    accuracy = correct_predictions / total_predictions * 100
    aft_softmax[choosed_exits]=accuracy

sorted(list(aft_softmax.items()), key=lambda x: x[1], reverse=True)

[((2, 3, 7, 9, 10), 88.44),
 ((3, 7, 9, 10), 88.42999999999999),
 ((1, 4, 6, 7, 8, 9, 10), 88.4),
 ((0, 3, 7, 9, 10), 88.39),
 ((4, 6, 7, 8, 9, 10), 88.38000000000001),
 ((0, 6, 9, 10), 88.37),
 ((2, 4, 6, 7, 8, 9, 10), 88.37),
 ((3, 7, 8, 9, 10), 88.36),
 ((1, 6, 7, 9, 10), 88.35),
 ((2, 7, 8, 9, 10), 88.35),
 ((0, 7, 9, 10), 88.34),
 ((5, 7, 9, 10), 88.34),
 ((3, 6, 7, 8, 9, 10), 88.34),
 ((1, 3, 6, 7, 8, 9, 10), 88.34),
 ((0, 1, 4, 6, 7, 8, 9, 10), 88.34),
 ((3, 6, 8, 10), 88.33),
 ((1, 3, 7, 9, 10), 88.33),
 ((1, 7, 8, 9, 10), 88.33),
 ((4, 7, 9, 10), 88.32),
 ((1, 3, 7, 8, 9, 10), 88.32),
 ((0, 3, 6, 7, 8, 9, 10), 88.32),
 ((3, 5, 7, 9, 10), 88.31),
 ((2, 3, 6, 7, 8, 9, 10), 88.31),
 ((0, 3, 7, 8, 9, 10), 88.3),
 ((0, 4, 6, 7, 8, 9, 10), 88.3),
 ((1, 6, 9, 10), 88.29),
 ((1, 7, 9, 10), 88.29),
 ((4, 6, 9, 10), 88.29),
 ((0, 6, 7, 9, 10), 88.29),
 ((1, 4, 6, 7, 9, 10), 88.29),
 ((3, 4, 6, 7, 8, 9, 10), 88.29),
 ((3, 7, 10), 88.28),
 ((2, 6, 9, 10), 88.28),
 ((1, 3, 6, 7, 9, 10), 88

##### 동적 앙상블: 모든 exit에서 특정 엔트로피 이하인 것들을 소프트맥스 전에 앙상블 하고 소프트맥스 진행 한 샘플에서 임계값보다 작은게 없으면 제일 뒤의 블록 선택만으로 추론

In [7]:
# 모든 exit에서 특정 엔트로피 이하인 것들을 모아서 앙상블을 해본다.
# case 2가지 소프트맥스 전에 합칠지 후에 합칠지;;

step_range = 10000
aft_sftmx = F.softmax(output_tensor,dim=2)
entropy_array= torch.tensor(entropy(aft_sftmx.to('cpu'), base=exit_num, axis=2))

min_entropy = entropy_array.min()
median_entropy = torch.median(torch.tensor(entropy_array))

step_size = (median_entropy - min_entropy) / step_range
d_bef_softmax=(0,0);d_aft_softmax=(0,0)
for mul in tqdm(range(step_range)):
    threshold = min_entropy + mul * step_size
    mask = (entropy_array[:, :] <= threshold).to(device)
    
    column_sums = mask.sum(dim=0)  
    zero_columns = (column_sums == 0)

    last_row = torch.zeros_like(mask)
    last_row[-1, :] = 1
    mask = mask | (last_row & zero_columns)

    mask.unsqueeze_(dim=2)
    masked_array=mask*output_tensor

    # sum before softmax (best acc: 87.75)
    ensemble_logits = masked_array.sum(axis=0)
    ensemble_probabilities = F.softmax(ensemble_logits, dim=1)
    _, predicted_labels = torch.max(ensemble_probabilities, dim=1)
    correct_predictions = (predicted_labels == labels).sum().item()
    total_predictions = labels.size(0)
    accuracy = correct_predictions / total_predictions * 100
    d_bef_softmax=max(d_bef_softmax,(threshold,accuracy),key=lambda x:x[1])

    # sum after softmax (best acc: 87.75)
    softmax_vector_list = F.softmax(masked_array, dim=2)
    ensemble_probabilities = softmax_vector_list.sum(axis=0)
    _, predicted_labels = torch.max(ensemble_probabilities, dim=1)
    correct_predictions = (predicted_labels == labels).sum().item()
    total_predictions = labels.size(0)
    accuracy = correct_predictions / total_predictions * 100
    d_aft_softmax=max(d_aft_softmax,(threshold,accuracy),key=lambda x:x[1])
print(f"d_bef_softmax: {d_bef_softmax}\nd_aft_softmax: {d_aft_softmax}")

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_23208\1631354111.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  median_entropy = torch.median(torch.tensor(entropy_array))
100%|██████████| 10000/10000 [00:08<00:00, 1198.87it/s]


d_bef_softmax: (tensor(0.1502), 88.18)
d_aft_softmax: (tensor(0.1207), 88.12)


##### 동적 앙상블: 모든 exit에서 특정 엔트로피 이하인 것들을 소프트맥스 전에 앙상블 하고 소프트맥스 진행 한 샘플에서 임계값보다 작은게 없으면 제일 엔트로피 작은 것을 선택해서 그것의 선택만으로 추론

In [8]:
# 모든 exit에서 특정 엔트로피 이하인 것들을 모아서 앙상블을 해본다.
# case 2가지 소프트맥스 전에 합칠지 후에 합칠지;;

step_range = 10000
aft_sftmx = F.softmax(output_tensor,dim=2)
entropy_array= torch.tensor(entropy(aft_sftmx.to('cpu'), base=exit_num, axis=2)).to(device)

min_entropy = entropy_array.min()
median_entropy = torch.median(torch.tensor(entropy_array))

step_size = (median_entropy - min_entropy) / step_range
d_bef_softmax=(0,0);d_aft_softmax=(0,0)

########################################
# 각 열의 최소값 인덱스 찾기
min_indices = entropy_array.argmin(dim=0)
# 2D 행렬 생성 및 초기화 (모든 값 0)
result_matrix = torch.zeros_like(entropy_array.int())
# 최소값 인덱스 위치에 1 설정
for col, row in enumerate(min_indices):
    result_matrix[row, col] = 1
########################################

for mul in tqdm(range(step_range)):
    threshold = min_entropy + mul * step_size
    mask = (entropy_array[:, :] <= threshold).to(device)
    
    column_sums = mask.sum(dim=0)
    zero_columns = (column_sums == 0)
    mask = mask | (result_matrix & zero_columns)

    mask.unsqueeze_(dim=2)
    masked_array=mask*output_tensor

    # sum before softmax (best acc: 87.75)
    ensemble_logits = masked_array.sum(axis=0)
    ensemble_probabilities = F.softmax(ensemble_logits, dim=1)
    _, predicted_labels = torch.max(ensemble_probabilities, dim=1)
    correct_predictions = (predicted_labels == labels).sum().item()
    total_predictions = labels.size(0)
    accuracy = correct_predictions / total_predictions * 100
    d_bef_softmax=max(d_bef_softmax,(threshold,accuracy),key=lambda x:x[1])

    # sum after softmax (best acc: 87.75)
    softmax_vector_list = F.softmax(masked_array, dim=2)
    ensemble_probabilities = softmax_vector_list.sum(axis=0)
    _, predicted_labels = torch.max(ensemble_probabilities, dim=1)
    correct_predictions = (predicted_labels == labels).sum().item()
    total_predictions = labels.size(0)
    accuracy = correct_predictions / total_predictions * 100
    d_aft_softmax=max(d_aft_softmax,(threshold,accuracy),key=lambda x:x[1])
print(f"d_bef_softmax: {d_bef_softmax}\nd_aft_softmax: {d_aft_softmax}")

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_23208\257576794.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  median_entropy = torch.median(torch.tensor(entropy_array))
100%|██████████| 10000/10000 [00:07<00:00, 1362.74it/s]

d_bef_softmax: (tensor(0.0705, device='cuda:0'), 88.01)
d_aft_softmax: (tensor(0.0700, device='cuda:0'), 88.0)


##### 동적 앙상블할 때 각 샘플들마다 어떤 출구들로 앙상블하는지 찍기 //시각화는 나중에

In [9]:
threshold = d_bef_softmax[0]
mask = (entropy_array[:, :] <= threshold).to(device)

indices = mask.nonzero()
column_indices = defaultdict(list)
for row, col in indices:
    column_indices[col.item()].append(row.item())

# 각 샘플별로 앙상블할 exit들의 인덱스를 저장
ensemble_exits_per_sample =[[] for _ in range(len(labels_list))]
for col in sorted(column_indices.keys()):
    ensemble_exits_per_sample[col] = column_indices[col]

USE_LAST_EXIT = 1
if USE_LAST_EXIT:
    # 앙상블할때 엔트로피가 임계값 이하인 exit가 하나도 없는 경우 마지막 exit의 추론 결과를 사용하도록 설정
    for i in range(len(labels_list)):
        if len(ensemble_exits_per_sample[i])==0:
            ensemble_exits_per_sample[i].append(exit_num-1) #10
else:
    # 엔트로피가 임계값 이하인 exit가 없는 경우, 가장 작은 엔트로피를 가진 exit 선택
    for i in range(len(labels_list)):
        if len(ensemble_exits_per_sample[i]) == 0:
            # i번째 열(sample)의 모든 엔트로피 값
            column_entropies = entropy_array[:, i]
            # 가장 작은 엔트로피를 가진 exit의 인덱스
            min_entropy_exit = torch.argmin(column_entropies)
            ensemble_exits_per_sample[i].append(min_entropy_exit.item())

ensemble_exits_per_sample

[[10],
 [10],
 [10],
 [3, 5, 6, 7, 8, 9, 10],
 [4, 5, 6, 7, 8, 9, 10],
 [10],
 [10],
 [10],
 [8, 9, 10],
 [10],
 [10],
 [10],
 [3, 6, 7, 10],
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 [2, 3, 4, 5, 6, 7, 8, 9, 10],
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 [5, 6, 7, 8, 9, 10],
 [2, 3, 4, 5, 6, 7, 8, 9, 10],
 [2, 3, 4, 5, 6, 7, 8, 9, 10],
 [3, 4, 5, 6, 7, 8, 9, 10],
 [3, 4, 5, 6, 7, 8, 9, 10],
 [10],
 [8, 10],
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 [3, 4, 5, 6, 7, 8, 9, 10],
 [10],
 [3, 4, 5, 6, 7, 8, 9, 10],
 [6, 9, 10],
 [2, 3, 10],
 [10],
 [6, 7, 8, 9, 10],
 [10],
 [5, 7, 8, 9, 10],
 [7, 8, 10],
 [6, 7, 8, 9, 10],
 [2, 3, 4, 7, 8, 9, 10],
 [3, 4, 10],
 [10],
 [6, 8, 9, 10],
 [7, 8, 9, 10],
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 [1, 2, 3, 4, 5, 6, 7],
 [10],
 [4, 5, 6, 7, 8, 9, 10],
 [10],
 [10],
 [3, 4, 5, 6, 7, 9, 10],
 [2, 3, 6, 7, 8, 9, 10],
 [0, 2, 3, 4, 5, 6, 7, 8, 9,

##### Temperature Scaling 한 각 출구들에서의 T 값

In [10]:
t_scalers_path = f"models/{data_choice}/temperature_scaler.pth"
t_scalers = torch.load(t_scalers_path)
t_scalers_values = torch.tensor([t_scalers[i].temperature for i in range(exit_num)]).to(device)
print(t_scalers_values)
t_scalers_values=t_scalers_values.unsqueeze_(dim=1).unsqueeze_(dim=2).expand_as(output_tensor)
ts_output_tensor=output_tensor/t_scalers_values

tensor([1.0655, 1.0274, 1.0515, 1.0217, 1.0312, 1.0137, 1.0590, 1.0306, 0.9919,
        0.9821, 1.3975], device='cuda:0')


##### TS동적 앙상블: 모든 exit에서 특정 엔트로피 이하인 것들을 앙상블 하고 소프트맥스 진행하고 맥스값으로 추론

In [11]:
# 모든 exit에서 특정 엔트로피 이하인 것들을 모아서 앙상블을 해본다.
# case 2가지 소프트맥스 전에 합칠지 후에 합칠지;;

step_range = 10000
aft_sftmx = F.softmax(ts_output_tensor,dim=2)
entropy_array= torch.tensor(entropy(aft_sftmx.to('cpu'), base=exit_num, axis=2))

min_entropy = entropy_array.min()
median_entropy = torch.median(torch.tensor(entropy_array))

step_size = (median_entropy - min_entropy) / step_range
d_bef_softmax=(0,0);d_aft_softmax=(0,0)
for mul in tqdm(range(step_range)):
    threshold = min_entropy + mul * step_size
    mask = (entropy_array[:, :] <= threshold).to(device)
    
    column_sums = mask.sum(dim=0)  
    zero_columns = (column_sums == 0)

    last_row = torch.zeros_like(mask)
    last_row[-1, :] = 1
    mask = mask | (last_row & zero_columns)

    mask.unsqueeze_(dim=2)
    masked_array=mask*output_tensor

    # sum before softmax (best acc: 87.75)
    ensemble_logits = masked_array.sum(axis=0)
    ensemble_probabilities = F.softmax(ensemble_logits, dim=1)
    _, predicted_labels = torch.max(ensemble_probabilities, dim=1)
    correct_predictions = (predicted_labels == labels).sum().item()
    total_predictions = labels.size(0)
    accuracy = correct_predictions / total_predictions * 100
    d_bef_softmax=max(d_bef_softmax,(threshold,accuracy),key=lambda x:x[1])

    # sum after softmax (best acc: 87.75)
    softmax_vector_list = F.softmax(masked_array, dim=2)
    ensemble_probabilities = softmax_vector_list.sum(axis=0)
    _, predicted_labels = torch.max(ensemble_probabilities, dim=1)
    correct_predictions = (predicted_labels == labels).sum().item()
    total_predictions = labels.size(0)
    accuracy = correct_predictions / total_predictions * 100
    d_aft_softmax=max(d_aft_softmax,(threshold,accuracy),key=lambda x:x[1])
print(f"d_bef_softmax: {d_bef_softmax}\nd_aft_softmax: {d_aft_softmax}")

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_23208\3092911740.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  median_entropy = torch.median(torch.tensor(entropy_array))
100%|██████████| 10000/10000 [00:08<00:00, 1198.79it/s]


d_bef_softmax: (tensor(0.0651), 87.98)
d_aft_softmax: (tensor(0.0651), 87.98)


##### TS동적 앙상블: 모든 exit에서 특정 엔트로피 이하인 것들을 앙상블 하고 소프트맥스 진행하고 맥스값으로 추론 임계값 이하가 없으면 제일 엔트로피 낮은거

In [12]:
# 모든 exit에서 특정 엔트로피 이하인 것들을 모아서 앙상블을 해본다.
# case 2가지 소프트맥스 전에 합칠지 후에 합칠지;;

step_range = 10000
aft_sftmx = F.softmax(ts_output_tensor,dim=2)
entropy_array= torch.tensor(entropy(aft_sftmx.to('cpu'), base=exit_num, axis=2)).to(device)

min_entropy = entropy_array.min()
median_entropy = torch.median(torch.tensor(entropy_array))

step_size = (median_entropy - min_entropy) / step_range
d_bef_softmax=(0,0);d_aft_softmax=(0,0)

########################################
# 각 열의 최소값 인덱스 찾기
min_indices = entropy_array.argmin(dim=0)
# 2D 행렬 생성 및 초기화 (모든 값 0)
result_matrix = torch.zeros_like(entropy_array.int())
# 최소값 인덱스 위치에 1 설정
for col, row in enumerate(min_indices):
    result_matrix[row, col] = 1
########################################

for mul in tqdm(range(step_range)):
    threshold = min_entropy + mul * step_size
    mask = (entropy_array[:, :] <= threshold).to(device)
    
    column_sums = mask.sum(dim=0)
    zero_columns = (column_sums == 0)
    mask = mask | (result_matrix & zero_columns)

    mask.unsqueeze_(dim=2)
    masked_array=mask*output_tensor

    # sum before softmax (best acc: 87.75)
    ensemble_logits = masked_array.sum(axis=0)
    ensemble_probabilities = F.softmax(ensemble_logits, dim=1)
    _, predicted_labels = torch.max(ensemble_probabilities, dim=1)
    correct_predictions = (predicted_labels == labels).sum().item()
    total_predictions = labels.size(0)
    accuracy = correct_predictions / total_predictions * 100
    d_bef_softmax=max(d_bef_softmax,(threshold,accuracy),key=lambda x:x[1])

    # sum after softmax (best acc: 87.75)
    softmax_vector_list = F.softmax(masked_array, dim=2)
    ensemble_probabilities = softmax_vector_list.sum(axis=0)
    _, predicted_labels = torch.max(ensemble_probabilities, dim=1)
    correct_predictions = (predicted_labels == labels).sum().item()
    total_predictions = labels.size(0)
    accuracy = correct_predictions / total_predictions * 100
    d_aft_softmax=max(d_aft_softmax,(threshold,accuracy),key=lambda x:x[1])
print(f"d_bef_softmax: {d_bef_softmax}\nd_aft_softmax: {d_aft_softmax}")

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_23208\4228968135.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  median_entropy = torch.median(torch.tensor(entropy_array))
100%|██████████| 10000/10000 [00:07<00:00, 1371.57it/s]

d_bef_softmax: (tensor(0.1794, device='cuda:0'), 87.15)
d_aft_softmax: (tensor(0.1794, device='cuda:0'), 87.11)


##### 각 출구마다 라벨 별로 Precision 확인 후 각 출구의 mAP 확인

In [13]:
exits_precision = np.array([[0.0] * dataset_outdim[data_choice] for _ in range(exit_num)])
for exit in [i for i in range(exit_num)]:
    label_correct = defaultdict(int)
    label_total = defaultdict(int)
    
    new_output_list = output_tensor[exit]
    logits = new_output_list
    probabilities = F.softmax(logits, dim=1)
    _, predicted = torch.max(probabilities, dim=1)
    for label, pred in zip(labels, predicted):
            if label == pred:
                label_correct[label.item()] += 1
            label_total[label.item()] += 1
    
    label_accuracy = {label: round(label_correct[label] / label_total[label] * 100, 2) for label in range(dataset_outdim[data_choice])}
    exits_precision[exit] = [label_accuracy[label] for label in range(dataset_outdim[data_choice])]


In [ ]:
exits_precision

array([[84., 67., 29., ..., 33., 19., 54.],
       [77., 77., 55., ..., 52., 32., 59.],
       [85., 86., 57., ..., 64., 49., 75.],
       ...,
       [95., 92., 67., ..., 91., 70., 92.],
       [96., 93., 72., ..., 96., 71., 92.],
       [87., 94., 67., ..., 94., 79., 91.]])

: 